# Whole declaration embedding

We ditch the chunking as we cannot get a satisfiying match.

## install libs

In [ ]:
!pip install datasets
!pip install langchain
!pip install langchain-groq
# !pip install -U sentence-transformers #check that session is in GPU mode #switch to openAI for a quick fix
!pip install openai
!pip install tiktoken # to count the tokens
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.9 MB/s eta 0

## dataset creation (adding embeddings)

In [ ]:
dataset_creation = True

In [ ]:
from datasets import load_dataset

if dataset_creation:
  declaration_ds = load_dataset("the-french-artist/hatvp_declarations_xml_plus_json_plus_index", split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
declaration_ds

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'declaration_json', 'text_index'],
    num_rows: 10944
})

In [ ]:
print(declaration_ds.select(range(1)).to_pandas().text_index.to_list()[0][:200])

damien abad -  député de l'ain


In [ ]:
# embed_df.head()

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def truncate_text_to_stay_under_openai_embedding_limit(input_text):

  backup_input_text = input_text

  openai_embed_limit = 8192
  delta = num_tokens_from_string(input_text, "cl100k_base") - openai_embed_limit
  while delta > 0:
    input_text = input_text[:-int(delta*2)] #we add factor 2 to speed up the process
    delta = num_tokens_from_string(input_text, "cl100k_base") - openai_embed_limit

  if len(input_text) > 0:
    return input_text
  else:
    return backup_input_text[:8000]

In [ ]:
from openai import OpenAI
client = OpenAI()
from tqdm.auto import tqdm
tqdm.pandas()


def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   text = truncate_text_to_stay_under_openai_embedding_limit(text)
   return client.embeddings.create(input = [text], model=model).data[0].embedding

get_embedding('this is a test')

In [ ]:
def get_embedding_for_map(row):
  row['text_index_embedding'] = get_embedding(row['text_index'])
  return row

In [ ]:
declaration_ds = declaration_ds.map(get_embedding_for_map, num_proc=2)

# TODO: batch 1k samples to go faster!!!

Parameter 'function'=<function get_embedding_for_map at 0x787b3ee7b760> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map (num_proc=2):   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
# embed_df['json_embedding'] = embed_df.progress_apply(lambda row: get_embedding(row['declaration_json']), axis=1)

In [ ]:
# embed_df

In [ ]:
# %%time
# from sentence_transformers import SentenceTransformer, util

# if dataset_creation:
#   #Load the model
#   # model = SentenceTransformer("intfloat/e5-mistral-7b-instruct") # NEEDS 26+GB VRAM WHICH GOOGLES WONT DELIVER.
#   model = SentenceTransformer("Alibaba-NLP/gte-Qwen1.5-7B-instruct") # STILL TOO MUCH VRAM NEEDED HERE!!!
#   model = SentenceTransformer("intfloat/multilingual-e5-large-instruct") #fingers crossed... NOOPE!


#   doc_emb = model.encode(embed_df.declaration_markdown.to_list())

In [ ]:
# if dataset_creation:
#   embed_df['markdown_embedding'] = list(doc_emb)

In [ ]:
# from datasets import Dataset

# if dataset_creation:
#   embed_ds = Dataset.from_pandas(embed_df)

In [ ]:
declaration_ds

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'declaration_json', 'text_index', 'text_index_embedding'],
    num_rows: 10944
})

In [ ]:
if dataset_creation:
  declaration_ds.push_to_hub("the-french-artist/hatvp_declarations_text_index_embeds")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/512 [00:00<?, ?B/s]

## dataset loading (no GPU needed)

In [ ]:
from datasets import load_dataset
embed_ds = load_dataset("the-french-artist/hatvp_declarations_text_index_embeds", split='train')

Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
# model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

In [ ]:
embed_ds.add_faiss_index(column='text_index_embedding')

  0%|          | 0/11 [00:00<?, ?it/s]

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'declaration_json', 'text_index', 'text_index_embedding'],
    num_rows: 10944
})

In [ ]:
import numpy as np

def perform_query(query, n_samples=1):
  # query_embed = model.encode([query])
  query_embed = np.array(get_embedding(query.lower()))
  scores, retrieved_examples = embed_ds.get_nearest_examples('text_index_embedding', query_embed, k=n_samples)
  return retrieved_examples['declaration_json']

In [ ]:
# embed_df = embed_ds.to_pandas()

In [ ]:
# potential_true_candidates = embed_df[embed_df['declaration_markdown'].str.contains('ABAD')].declaration_markdown.to_list()
# for curr_candidate in potential_true_candidates:
#   display_markdown(curr_candidate, raw=True)

In [ ]:
import json

def get_name_surname_from_str_declaration(input_str_json):
  parsed_json = json.loads(input_str_json)
  return parsed_json['declaration']['general']['declarant']['nom'], parsed_json['declaration']['general']['declarant']['prenom']

In [ ]:
results = perform_query("Qui est Damien Abad ?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('ABAD', 'DAMIEN')
('ABAD', 'DAMIEN')
('ADAM', 'Damien')
('ADAM', 'Damien')
('Delavoie', 'Damien')
('STEPHO', 'Damien')
('STEPHO', 'Damien')
('DEMARTHE', 'Pascal')
('DEMARTHE', 'Pascal')
('DEMARTHE', 'Pascal')


In [ ]:
results = perform_query("Quel est le salaire de Damien Abad en 2019 ?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('ABAD', 'DAMIEN')
('ABAD', 'DAMIEN')
('ADAM', 'Damien')
('ADAM', 'Damien')
('STEPHO', 'Damien')
('STEPHO', 'Damien')
('DEMARTHE', 'Pascal')
('DEMARTHE', 'Pascal')
('DEMARTHE', 'Pascal')
('HUGUET', 'Damien')


In [ ]:
results = perform_query("Qui est Damien ABAD? Et quel est son salaire en 2019?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('ABAD', 'DAMIEN')
('ABAD', 'DAMIEN')
('ADAM', 'Damien')
('ADAM', 'Damien')
('STEPHO', 'Damien')
('STEPHO', 'Damien')
('Charlet', 'Damien')
('Charlet', 'Damien')
('Charlet', 'Damien')
('Charlet', 'Damien')


In [ ]:
results = perform_query("Qui est Antoine ARMAND?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('Armand', 'Antoine')
('Armand', 'Antoine')
('Armand', 'Antoine')
('JEAN', 'Antoine')
('JEAN', 'Antoine')
('Mokrane', 'Antoine')
('BACK', 'Antoine')
('Guillou', 'Antoine')
('Guillou', 'Antoine')
('Guillou', 'Antoine')


In [ ]:
results = perform_query("Qui est un Député de Haute-Savoie?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('ROSEREN', 'xavier')
('ROSEREN', 'xavier')
('Armand', 'Antoine')
('Armand', 'Antoine')
('Armand', 'Antoine')
('Petex', 'Christelle')
('Petex', 'Christelle')
('HERVE', 'loic')
('HERVE', 'loic')
('Vial', 'Cédric')


In [ ]:
results = perform_query("Qui est une Députée de Haute-Savoie?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('Petex', 'Christelle')
('Petex', 'Christelle')
('Duby-Muller', 'Virginie')
('Duby-Muller', 'Virginie')
('Duby-Muller', 'Virginie')
('Violland', 'Anne-Cécile')
('Violland', 'Anne-Cécile')
('RIOTTON', 'Véronique')
('RIOTTON', 'Véronique')
('NOEL', 'Sylviane')


In [ ]:
results = perform_query("Le conjoint de Antoine ARMAND est il un homme ou une femme?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('Armand', 'Antoine')
('Armand', 'Antoine')
('Armand', 'Antoine')
('BACK', 'Antoine')
('Guillou', 'Antoine')
('Guillou', 'Antoine')
('Guillou', 'Antoine')
('Armanet', 'Guy')
('Mokrane', 'Antoine')
('SENANT', 'Jean-Yves')


In [ ]:
results = perform_query("Quel est l'ensemble des revenus perçus par Antoine ARMAND?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('Armand', 'Antoine')
('Armand', 'Antoine')
('Armand', 'Antoine')
('JEAN', 'Antoine')
('JEAN', 'Antoine')
('Léaument', 'Antoine')
('Léaument', 'Antoine')
('SENANT', 'Jean-Yves')
('SENANT', 'Jean-Yves')
('SENANT', 'Jean-Yves')


In [ ]:
results = perform_query("Le nom du déclarant est Antoine ARMAND?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('Armand', 'Antoine')
('Armand', 'Antoine')
('Armand', 'Antoine')
('Mokrane', 'Antoine')
('Léaument', 'Antoine')
('Léaument', 'Antoine')
('JEAN', 'Antoine')
('JEAN', 'Antoine')
('VERMOREL', 'ANTOINE')
('VERMOREL', 'ANTOINE')


In [ ]:
results = perform_query("Qui est le maire de Bordeaux?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('BOUTLEUX', 'Dimitri')
('BOUTLEUX', 'Dimitri')
('GUILLEMIN', 'Laurent')
('CAZAUX', 'OLIVIER')
('CAZAUX', 'OLIVIER')
('BLANC', 'BERNARD')
('BLANC', 'BERNARD')
('HAZOUARD', 'Mathieu')
('HAZOUARD', 'Mathieu')
('JEANJEAN', 'Didier')


In [ ]:
results = perform_query("Qui est maire de Bordeaux?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('BOUTLEUX', 'Dimitri')
('BOUTLEUX', 'Dimitri')
('GUILLEMIN', 'Laurent')
('CAZAUX', 'OLIVIER')
('CAZAUX', 'OLIVIER')
('Bouisson', 'Dominique')
('HAZOUARD', 'Mathieu')
('BLANC', 'BERNARD')
('BLANC', 'BERNARD')
('HAZOUARD', 'Mathieu')


In [ ]:
results = perform_query("Qui est pierre hurmic?", 10)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('HURMIC', 'Pierre')
('HURMIC', 'Pierre')
('HURMIC', 'Pierre')
('HURMIC', 'Pierre')
('MICHEL', 'Patrick')
('MICHEL', 'Patrick')
('MICHEL', 'Pierre')
('MICHEL', 'Pierre')
('Huguet', 'Pierre')
('Huguet', 'Pierre')


In [ ]:
# from IPython.display import display_markdown

# results = perform_query("Qui est Damien Abad ?")

# for result in results:
#   display_markdown(
#       result,
#       raw=True)

In [ ]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [ ]:
def get_answer_to_question_no_context(question, llm_to_use):

  system = """You are an assistant for question-answering tasks.
  Use three sentences maximum and keep the answer concise.
  """
  human = "{text}"
  prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
  actual_prompt = f"""
  Question: {question}
  Answer:
  """

  chain = prompt | llm_to_use | StrOutputParser()
  return chain.invoke({"text": actual_prompt})

In [ ]:
def get_RAG_request(question, llm_to_use):

  system = """You are an assistant for document retrieval tasks.
  You have access to a database of entries related to french politicians.
  Each entry is identified by a name and a surname.
  When provided with a question, return the name and surname of the most probable data entry.
  Only respond with the name and surname.

  """
  human = "{text}"
  prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
  actual_prompt = f"""
  Question: {question}
  Answer:
  """

  chain = prompt | llm_to_use | StrOutputParser()
  return chain.invoke({"text": actual_prompt})

In [ ]:
def get_answer_to_question(question, llm_to_use):

  system = """You are an assistant for question-answering tasks.
  Use the following pieces of retrieved context to answer the question.
  If you don't know the answer, just say that you don't know.
  Use three sentences maximum and keep the answer concise.
  """
  human = "{text}"
  prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

  results = perform_query(question, 1)
  context = ''.join(results) #concatenate top 5 results into a context
  actual_prompt = f"""
  Question: {question}
  Context: {context}
  Answer:
  """

  # print(context)
  for result in results:
    print(get_name_surname_from_str_declaration(result))
  chain = prompt | llm_to_use | StrOutputParser()
  return chain.invoke({"text": actual_prompt})

In [ ]:
def get_answer_to_question_improved(question, llm_to_use):

  system = """You are an assistant for question-answering tasks.
  Use the following pieces of retrieved context to answer the question.
  If you don't know the answer, just say that you don't know.
  Use three sentences maximum and keep the answer concise.
  """
  human = "{text}"
  prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

  retrieval_question = get_RAG_request(question, llm_to_use)
  print(f"Retrieval question : {retrieval_question}")
  print()
  results = perform_query(retrieval_question, 1)
  context = ''.join(results) #concatenate top 5 results into a context
  actual_prompt = f"""
  Question: {question}
  Context: {context}
  Answer:
  """

  print("Most relevant context(s) : ")
  for result in results:
    print(get_name_surname_from_str_declaration(result))
  print()
  chain = prompt | llm_to_use | StrOutputParser()
  return chain.invoke({"text": actual_prompt})

In [ ]:
question = "Qui est Damien ABAD? Et quel est son salaire en 2019?"

In [ ]:
results = perform_query(question, 1)
for result in results:
  print(get_name_surname_from_str_declaration(result))

('ABAD', 'DAMIEN')


In [ ]:
# results

In [ ]:
mixtral_llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")
llm_llama3_70B = ChatGroq(temperature=0, model_name="llama3-70b-8192")

get_answer_to_question(question, llm_llama3_70B)

('ABAD', 'DAMIEN')


'Damien Abad is a French politician, and his salary in 2019 was €71,105.'

In [ ]:
llm_llama3_8B = ChatGroq(temperature=0, model_name="llama3-8b-8192")
get_answer_to_question(question, llm_llama3_8B)

('ABAD', 'DAMIEN')


"Damien Abad's salary in 2019 was 71,105 euros."

In [ ]:
print(get_answer_to_question("Quel est l'ensemble des revenus perçus par Damien Abad?", llm_llama3_8B))

('ABAD', 'DAMIEN')
The ensemble des revenus perçus par Damien Abad includes:

* Rémunération de député : 67 047 € en 2017, 71 042 € en 2018, 71 105 € en 2019, 70 773 € en 2020, 70 676 € en 2021 et 27 289 € en 2022.
* Rémunération de conseiller départemental : 28 007 € en 2017, 24 201 € en 2018, 16 386 € en 2019, 16 386 € en 2020, 16 384 € en 2021 et 6 827 € en 2022.
* Rémunération de président du groupe LR à l'Assemblée nationale : 0 € en 2019, 0 € en 2020, 0 € en 2021 et 0 € en 2022.
* Rémunération de président du SDIS 01 : 0 € en 2015, 0 € en 2016 et 0 € en 2017.
* Rémunération de président d'Aintourisme : 0 € en 2017, 0 € en 2018, 0 € en 2019, 0 € en 2020, 0 € en 2021 et 0 € en 2022.
* Rémunération de président du groupe LR à l'Assemblée nationale : 0 € en 2019, 0 € en 2020, 0 € en 2021 et 0 € en 2022.
* Rémunération de président de l'association Saveurs de l'Ain : 0 € en 2019, 0 € en 2020, 0 € en 2021 et 0 € en 2022.
* Rémunération de président de l'association Les Amis de Damien A

In [ ]:
print(get_answer_to_question("Où travaille le conjoint de Damien Abad?", llm_llama3_8B))

('ABAD', 'DAMIEN')
Le conjoint de Damien Abad travaille au Centre Hospitalier du Haut-Bugey.


In [ ]:
print(get_answer_to_question("Où travaille le conjoint de Damien Abad?", llm_llama3_70B))

('ABAD', 'DAMIEN')
Le conjoint de Damien Abad travaille au Centre Hospitalier du Haut-Bugey en tant qu'infirmière.


In [ ]:
print(get_answer_to_question("Où travaille le conjoint de Damien Abad?", mixtral_llm))

('ABAD', 'DAMIEN')
The spouse of Damien Abad works as an "Infirmière" (nurse) at the "CENTRE HOSPITALIER DU HAUT-BUGEY".


In [ ]:
print(get_answer_to_question("Le conjoint de Damien Abad est il un homme ou une femme?", llm_llama3_8B))

('ABAD', 'DAMIEN')
Le conjoint de Damien Abad est une femme.


In [ ]:
print(get_answer_to_question("Le conjoint de Damien Abad est il un homme ou une femme?", llm_llama3_70B))

('ABAD', 'DAMIEN')
Le conjoint de Damien Abad est une femme, car il est mentionné que son conjoint est une infirmière.


In [ ]:
print(get_answer_to_question("Le conjoint de Damien Abad est il un homme ou une femme?", mixtral_llm))

('ABAD', 'DAMIEN')
The context does not provide information on Damien Abad's spouse's gender. The provided document focuses on Damien Abad's professional activities and declarations of interests.


In [ ]:
print(get_answer_to_question("Où travaille le conjoint de Loïc HERVÉ le sénateur de Haute Savoie?", mixtral_llm))

('HERVE', 'loic')
According to the provided context, Loïc HERVÉ's spouse is an Attachée principale (FPT, catégorie A) and works for Collectivités locales.


In [ ]:
print(get_answer_to_question("Où travaille le conjoint de Antoine Armand?", mixtral_llm))

('Armand', 'Antoine')
The context provided does not include information about the spouse of Antoine Armand or their place of work.


In [ ]:
print(get_answer_to_question("Où travaille le conjoint de Eric PARRA", llm_llama3_70B))

('PARRA', 'ERIC')
Le conjoint d'Eric PARRA travaille comme assistante de vie.


In [ ]:
print(get_answer_to_question("Quel est le plus récent emploi d'Eric PARRA? Et sa rémunération?", mixtral_llm))

('PARRA', 'ERIC')
Eric PARRA's most recent job is as an assistant of life. His remuneration for this position in 2020 was 7,000. However, if you're asking about his remuneration for his mandates, he received 6,650 as Vice President of the Communauté d'Agglomération du Grand Narbonne and 1,545 as Conseller Municipal of the Ville de Narbonne in 2020.


In [ ]:
print(get_answer_to_question("Combien gagne le maire de Bordeaux?", llm_llama3_70B))

('BOUTLEUX', 'Dimitri')
The question asks how much the mayor of Bordeaux earns. However, the provided context does not explicitly mention the salary of the mayor of Bordeaux. It appears to be a declaration of interests and assets of a municipal councilor or mayor, but the salary information is not present.


In [ ]:
get_answer_to_question_no_context("Qui est le maire de Bordeaux", mixtral_llm)

'The current mayor of Bordeaux, France is Pierre Hurmic. He took office on July 4, 2020. He is a member of the Ecologist Party.'

In [ ]:
get_answer_to_question_no_context("Combien gagne le maire de Bordeaux", mixtral_llm)

'The mayor of Bordeaux, Pierre Hurmic, earns a salary of around 5,250 euros net per month. This amount can vary depending on the deductions and contributions. His salary is made public according to the transparency law in France.'

In [ ]:
get_RAG_request("Qui est le maire de Bordeaux?", mixtral_llm)

'Pierre Hurmic'

In [ ]:
get_RAG_request("Combien gagne le maire de Bordeaux", mixtral_llm)

'Pierre Hurmic'

In [ ]:
get_answer_to_question_improved("Combien d'argent est payé le maire de Bordeaux?", mixtral_llm)

Retrieval question : Pierre Hurmic

Most relevant context(s) : 
('HURMIC', 'Pierre')



'Based on the provided context, the document is a declaration of interests for Pierre Hurmic, who was a Vice President of Bordeaux Métropole as of July 17, 2020. There is no information about the salary of the mayor of Bordeaux in this document.'

In [ ]:
embed_df = embed_ds.to_pandas()

In [ ]:
embed_df[embed_df['text_index'].str.contains('hurmic')]

,xml_sha1,declaration_xml,declaration_json,text_index,text_index_embedding
5348,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>03/09/2020 17:42:05</d...,"{""declaration"": {""dateDepot"": ""03/09/2020 17:4...",pierre hurmic - maire de bordeaux,"[0.007317204028367996, 0.0006617681938223541, ..."
5349,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>03/09/2020 17:44:15</d...,"{""declaration"": {""dateDepot"": ""03/09/2020 17:4...",pierre hurmic - maire de bordeaux,"[0.007317204028367996, 0.0006617681938223541, ..."
5350,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>08/04/2021 19:10:43</d...,"{""declaration"": {""dateDepot"": ""08/04/2021 19:1...",pierre hurmic - maire de bordeaux,"[0.007317204028367996, 0.0006617681938223541, ..."
5351,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>25/05/2021 19:38:47</d...,"{""declaration"": {""dateDepot"": ""25/05/2021 19:3...",pierre hurmic - maire de bordeaux,"[0.007317204028367996, 0.0006617681938223541, ..."


In [ ]:
embed_df[embed_df['text_index'].str.contains('BOUTLEUX'.lower())]

,xml_sha1,declaration_xml,declaration_json,text_index,text_index_embedding
1596,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>26/07/2020 18:03:16</d...,"{""declaration"": {""dateDepot"": ""26/07/2020 18:0...",dimitri boutleux - adjoint au maire de bordeaux,"[-0.017299488186836243, -0.014234055764973164,..."
1597,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>02/08/2021 10:43:06</d...,"{""declaration"": {""dateDepot"": ""02/08/2021 10:4...",dimitri boutleux - adjoint au maire de bordeaux,"[-0.017299488186836243, -0.014234055764973164,..."
1598,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>05/07/2021 13:25:38</d...,"{""declaration"": {""dateDepot"": ""05/07/2021 13:2...",guy boutleux - vice-président de la communaut...,"[-0.011403901502490044, -0.025366488844156265,..."


# CONCLUSION OF THE MARKDOWN TEST:


- the openai embedding system is very slow (1H for 10k embeddings) but at least it works, unlike the open source lighter ones (in french that is).  

- we cannot get interesting information because the retrieval query is very different from the actual query. Eg. What is XXX salary in 2019? The retrieval query will be "get me XXX data" while the generative query is the input one.  

- we are missing information from the markdown files: after 3 days of intensive attempts to make an SQL, then simple Markdown file, from the input XML files, we still cannot ensure that some entire sections are not missing due to the fickle nature of the context


## What to do next:  

- find a way to get embeddings faster and cheaper: we need to run the system in a dedicated notebook with enough VRAM, using a quantized version I suppose (given that we do not have enough VRAM, either locally or online)  
- Make a new dataset from the XML base declarations that simply converts XML to JSON => this will make the context lighter while not need to have massive OO classes that are not able to parse the thing, even after writing more than 250 if statements on "None".  
- find a way to execute online : this system will still need a local FAISS index and/or a local embedding for the dataset creation, as well as an acess to an LLM for retrieval => can we get free (or near free) online stuff to perform these operations? (Groq is already a good thing).  

# CONCLUSION OF THE JSON TEST

We have complete categories as we cannot possibly fail (we are using XMLTODICT lib to convert XML to JSON).  
However, this doesn't seem to prevent problems:  
- we cannot match a given person declaration! Even when setting up the query perfectly, we are in 5th position, with 4 irrelevant matches before. Even after analyzing all semantically related text, we cannot explain this weird occurance...
- this use of JSON instead of XML decreases char count by an average of 25%; which is nice but not enough to use several declarations in a single context request (only Mixtral was able to process it).  

````
RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hw5634epebfvhswdt88pdsm3` on tokens per minute (TPM): Limit 7000, Used 0, Requested ~7485. Please try again in 4.157142857s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
````

## What to do next

- extract text from JSON so the embeddings are les confused by keys maybe?  
- we noticed a change in performance between ALLCAPS and alllower so we will preventively set all text to lowerchars and process all queries to lower chars so this problem is dealt with.
- recompute embeddings and test results.  

# CONCLUSION OF THE TEXT TEST

We have better retrieval rates: Damien Abad is perfectly recognized in our tests. However, Antoine Armand is not, at all. It seems that we can also retrieve people by their title, like "Député de la Haute-Savoie", but not perfectly, as many will in fact be "Senateurs" or "Député de Savoie" instead.  

We still rely on full JSON inclusion in the context as the text only version is too cryptic for an LLM to analyze its contents.  

## What to do next

- CHUNKING! We will chunk each declaration into large subsections (the main keys under "declaration") and prefix them with the name and work title of the person making the declaration. This way, we will be able to ask precise questions about something related to someone.  
- Embedding: it seems that we can pass up to 2048 embedding requests at once (see https://cookbook.openai.com/examples/embedding_wikipedia_articles_for_search ) so we should do that to handle the increased needs due to chunking.  

This "chunking with person name+title prefix" technique will improve questions about someone, but still not solve the problem of aggregate questions like:  
- who is making more than X in year Y ?  
- list people working at place X ?
- list people affiliated with party X ?
etc...

These will only get solved by using SQL and a structured database.  

# CONCLUSION OF THE TEXT WITH METADATA TEST

It works even better! Now we get 90% succes when naming someone in a query, and we get consistent results.  
The only shortcoming is when several people have the same name, either same name+surname, or a shared name as a surname. We can disimbiguate using the current job title, although that is only possible by doing some research.  
The special chars are also a problem, their usage is inconsistent in the declarations, eg. Loic VS Loïc.  

- adding a name+surname+job title prefix to the context is a great success
- we should test the job titles more deeply to see how to exploit them  
- the advanced questions are also a great success due to the complete declaration being passed in the context  

## What to do next  
- find a way to embed a link to the current declaration so our answers come with a URL to fact check the answers  
- convert the simple RAG into a discussion by passing the chat history

# CONCLUSION OF THE TEXT-INDEX TEST

Best results so far, we have a perfect match when asking for a given person.  
Now, the problem seems to be the quality of the job titles: XML files contain hazy information, this is not great for creating the index, we will have to rely on a seconday data source to improve this.  

- the persons name and surname is a perfect match
- we can ask complex questions and still match the correct context

## What to do next  

- load the extented information (.csv file with actual job titles) and re-create the index from there  
- do a second test  

# CONCLUSION OF THE SECOND TEXT-INDEX TEST  

Afterr retrieval of the correct job titles, we still cannot get a positive match for the mayor of Bordeaux.  
Sadly, the second nin chief is always recognized first.  
We then tried making a second guess by first having an LLM ask a retrieval question, then have a final question based on the "higher" retrieval quality.  

It turns out that the dumb LLM (mixtral) gets the correct answer to the mayor question and returns the correct context, but then, is incapable of answering correctly.  
The "smart" llm (LLama3) cannot get the correct retrieval query (asks about the previous maire...the change dates back to 2019 when the cutoff date is supposed to be 2023, and the LLM gives the correct answer on huggingface, so Groq might be quantizing a lot here...) so the test fails.  

- we have correct identification results, although the exact synthax (eg. a `d'` VS `de` before the name) seems to have a big impact.  
- we cannot ask general questions about job titles as a job title containing part of another job title will weight more (no job title hierarchy knowledge in the embedding). Eg. `mayor helper` will be ranked higher than `mayor` when looking for the maire.  

## What to do next  

- we should use better quality LLMs to ask retrieval questions, as we do not pass documents in the context => the requests will be quite cheap.  
- we can add more information in the index? No ideas for now.  